In [3]:
from quant_rotor.core.sparse.hamiltonian import hamiltonian_sparse
from quant_rotor.core.dense.hamiltonian import hamiltonian_dense

from quant_rotor.core.dense.hamiltonian_big import hamiltonian_general_dense
from quant_rotor.core.sparse.hamiltonian_big import hamiltonian_general_sparse
from quant_rotor.core.dense.de_solve_one_thermal import integration_scheme
import numpy as np
from quant_rotor.models.sparse.support_ham import build_V_in_p
import matplotlib.pyplot as plt
from quant_rotor.models.dense.density_matrix import density_matrix_1
import quant_rotor.models.dense.thermofield_boltz_funcs as bz
import opt_einsum as oe

from scipy.optimize import minimize

In [4]:
np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=np.inf)
np.set_printoptions(threshold=np.inf)

# Check TF

In [ ]:
site = 3
state = 3
g = 0.1

In [ ]:
H, K, V = hamiltonian_dense(state, site, g)

In [ ]:
state_2 = state**2

In [ ]:
U, _ = bz.thermofield_change_of_basis(K)

I = np.eye(state)

K_prim = oe.contract('pq,mw->pmqw', K, I,  optimize='optimal').reshape(state_2, state_2)

K_tilda = U.T @ K_prim @ U

V_tensor = V.reshape(state, state, state, state)

V_prim = oe.contract('pqrs,mw,nv->pmqnrwsv', V_tensor, I, I,  optimize='optimal').reshape(state_2**2, state_2**2)

V_grouped = V_prim.reshape(state**2, state**2, state**2, state**2)

V_tilda = oe.contract('Mi,Wj,ijab,aN,bV->MWNV', U.T, U.T, V_grouped, U, U, optimize='optimal').reshape(state_2**2, state_2**2)

In [ ]:
V_tilda.shape

In [ ]:
H_TF_prim, _, _ = hamiltonian_dense(state**2, site, g, K_import=K_prim, V_import=V_prim, Import=True)
H_TF, _, _ = hamiltonian_dense(state**2, site, g, K_import=K_tilda, V_import=V_tilda, Import=True)

In [ ]:
eig_val_TF_prim, eig_vec_TF_prim = np.linalg.eigh(H_TF_prim)
eig_val_TF, eig_vec_TF = np.linalg.eigh(H_TF)
eig_val, eig_vec = np.linalg.eigh(H)

In [ ]:
index_OR = np.argsort(-eig_val)
index_TF = np.argsort(-eig_val_TF)
index_TF_prim = np.argsort(-eig_val_TF_prim)

In [ ]:
print(eig_val[index_OR])
print(eig_val_TF[index_TF])
print(eig_val_TF[index_TF_prim])

In [ ]:
eig_val_TF[index_TF]

In [ ]:
psi_vec = eig_vec_TF[index_TF[9]]

rho_site_0 = density_matrix_1(state_2, site, psi_vec, 0)

In [ ]:
eig_val_D, matrix_p_to_NO_full = np.linalg.eigh(rho_site_0)
index = np.argsort(-eig_val_D)

In [ ]:
eig_val_D[index]

# Sparse

In [ ]:
site = 3
state = 3
g = 0.9

In [ ]:
H, K, V = hamiltonian_general_sparse(state, site, g)

K = K.toarray()
V = V.toarray()

In [ ]:
U, _ = bz.thermofield_change_of_basis(K)

K_tilda = bz.H_tilde_maker(K)

V_tensor = V.reshape(state, state, state, state)

I = np.eye(state)
V_prim = oe.contract('pqrs,mw,nv->pmqnrwsv', V_tensor, I, I,  optimize='optimal')

V_grouped = V_prim.reshape(state**2, state**2, state**2, state**2)

V_tilda = oe.contract('Mi,Wj,ijab,aN,bV->MWNV', U, U, V_grouped, U, U, optimize='optimal')

In [ ]:
state_2 = state**2

K_tilda_s = sp.csr_matrix(K_tilda)
V_tilda_s = sp.csr_matrix(V_tilda.reshape(state_2**2, state_2**2))

In [ ]:
H_TF, _, _ = hamiltonian_sparse(state_2, site, g, K_import=K_tilda, V_import=V_tilda)

In [ ]:
eig_val_D, matrix_p_to_NO_full = np.linalg.eigh(rho_site_0)
index = np.argsort(-eig_val_D)

In [ ]:
_, psi_vec = sp.linalg.eigsh(H_TF, k=1, which='SA', tol=1e-19, maxiter=2000)

rho_site_0 = density_matrix_1(state_2, site, psi_vec, 0)

In [ ]:
eig_val_D, matrix_p_to_NO_full = np.linalg.eigh(rho_site_0)
index = np.argsort(-eig_val_D)

In [ ]:
eig_val_D[index[:2]]

# Dense

In [5]:
site = 3
state = 3
g = 0.5

In [6]:
H, K, V = hamiltonian_general_dense(state, site, g)

In [7]:
state_2 = state**2

In [8]:
U, _ = bz.thermofield_change_of_basis(K)

K_tilda = bz.H_tilde_maker(K)

V_tensor = V.reshape(state, state, state, state)

I = np.eye(state)
V_prim = oe.contract('pqrs,mw,nv->pmqnrwsv', V_tensor, I, I,  optimize='optimal')

V_grouped = V_prim.reshape(state**2, state**2, state**2, state**2)

V_tilda = oe.contract('Mi,Wj,ijab,aN,bV->MWNV', U.T, U.T, V_grouped, U, U, optimize='optimal').reshape(state_2**2, state_2**2)

In [9]:
H_TF, _, _ = hamiltonian_dense(state_2, site, g, K_import=K_tilda, V_import=V_tilda, Import=True)

In [10]:
eig_val_TF, eig_vec_TF = np.linalg.eigh(H_TF)

In [11]:
index_TF = np.argsort(-eig_val_TF)
psi_vec = eig_vec_TF[index_TF[0]]

rho_site_0 = density_matrix_1(state_2, site, psi_vec, 0)

In [12]:
eig_val_D, matrix_p_to_NO_full = np.linalg.eigh(rho_site_0)
index = np.argsort(-eig_val_D)

In [13]:
eig_val_D[index]

array([ 0.65464,  0.30298,  0.04238,  0.     ,  0.     ,  0.     ,  0.     , -0.     , -0.     ])

In [14]:
matrix_p_to_NO_full[index[0]].real

array([ 0.     ,  0.     ,  0.     ,  0.     ,  0.     , -0.     ,  0.03703,  0.99931, -0.     ])

# Optimization of the degenerate states

## Constructing and testing transition density matrix

In [15]:
def transition_density_matrix_1(states: int, sites: int, eigvector_1: np.ndarray, eigvector_2: np.ndarray, choose_site: int) -> np.ndarray:

    D_1 = np.zeros((states, states), dtype=complex)

    n_lambda = states**(choose_site)
    n_mu = states**(sites - choose_site - 1)

    psi1 = eigvector_1.reshape(n_lambda, states, n_mu)
    psi2 = eigvector_2.reshape(n_lambda, states, n_mu)

    # D[p, p'] = sum_{lambda, mu} psi1^*(lambda, p, mu) * psi2(lambda, p', mu)
    D = np.einsum('lpm,lqm->pq', psi1, psi2, optimize=True)

    return D

In [16]:
ground_vec_1_TF = eig_vec_TF[index_TF[0]]
ground_vec_2_TF = eig_vec_TF[index_TF[1]]

In [17]:
eig_val, eig_vec = np.linalg.eigh(H)

index_OR = np.argsort(-eig_val)

In [18]:
ground_vec_1_OR = eig_vec[index_OR[0]]

D_1 = density_matrix_1(state, site, ground_vec_1_OR, 0)

In [19]:
D_t = transition_density_matrix_1(state, site, ground_vec_1_OR, ground_vec_1_OR, 0)

In [20]:
eig_val[index_OR]

array([ 3.92045,  3.77226,  3.42491,  3.42491,  3.39718,  3.39718,  3.21992,  3.19802,  3.19802,  3.11202,  3.11202,  3.00694,  3.00618,  2.66112,  1.9371 ,  1.9371 ,  1.60736,  1.57041,  1.57041,  1.38151,  1.02876,  0.64545,  0.64545,  0.16503,  0.16503, -1.91798, -2.01129])

In [21]:
np.trace(D_t @ D_t)

np.complex128(0.6419012396579806+0j)

## Definde maximization function

In [22]:
ground_NO = eig_vec_TF[:, index_TF[:27]]

In [23]:
ground_NO.shape

(729, 27)

In [37]:
def f_max(C, ground_NO):
    # C is variable; ground_NO is fixed from args
    psi = ground_NO @ C
    D = density_matrix_1(state*2, site, psi, 0)
    return float(np.real(np.trace(D)**2))

# inequality constraint: f_max(C, ground_NO) <= 1  →  1 - f_max >= 0
def constr_ineq(C, ground_NO):
    return 1 - f_max(C, ground_NO)

# optional equality constraint: <psi|psi> = 1
def constr_eq(C):
    return float(C @ C) - 1.0

# initial guess
C0 = np.full((state**2*site), 1e-10)

res = minimize(
    constr_ineq,
    x0=C0,
    args=(ground_NO,),                       # <-- fixed extra argument(s)
    method='SLSQP',
    constraints=[
        {'type': 'eq', 'fun': constr_eq}
    ],
    options={'ftol': 1e-6, 'maxiter': 1000}
)

In [ ]:
# res = minimize(
#     constr_ineq, C0, args=(ground_NO,),
#     method='trust-constr',    # ← different method
#     constraints=[{'type': 'eq', 'fun': constr_eq}],
#     options={'gtol':1e-8, 'maxiter':10000}
# )

/Users/gilfrim/Desktop/Projects/Computational_Analysis_of_Many_Rotor_Systems/.venv/lib/python3.13/site-packages/scipy/optimize/_differentiable_functions.py:552: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


In [38]:
res

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.17004222298246663
       x: [-7.664e-03 -2.985e-02 ...  2.885e-01  6.604e-02]
     nit: 14
     jac: [ 2.545e-02  9.908e-02 ... -9.576e-01 -2.192e-01]
    nfev: 407
    njev: 14

In [39]:
C_opt = res.x

In [40]:
C_opt @C_opt

np.float64(1.0000000302379362)

In [41]:
psi = ground_NO @ C_opt

In [42]:
D_1 = density_matrix_1(state**2, site, psi, 0)

In [43]:
eig_val_TF_D, eig_vec_TF_D = np.linalg.eigh(D_1)
index = np.argsort(-eig_val_TF_D)

In [44]:
constr_ineq(C_opt, ground_NO)

0.17004222298246663

In [45]:
eig_val_TF_D[index]

array([0.91386, 0.05206, 0.03408, 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ])

In [34]:
eig_vec_TF_D

array([[-0.51639+0.j,  0.24843+0.j, -0.22268+0.j, -0.53731+0.j,  0.     +0.j,  0.     -0.j,  0.     +0.j,  0.     +0.j, -0.57735-0.j],
       [-0.64962+0.j,  0.02534+0.j, -0.27798+0.j, -0.00855+0.j, -0.     +0.j, -0.     +0.j,  0.     +0.j, -0.     +0.j,  0.70711+0.j],
       [-0.39489+0.j, -0.30744+0.j, -0.16657+0.j,  0.74507+0.j,  0.     +0.j,  0.     +0.j,  0.     +0.j, -0.     +0.j, -0.40825+0.j],
       [-0.     +0.j, -0.     +0.j,  0.     +0.j, -0.     +0.j,  0.     +0.j,  0.     +0.j,  0.     +0.j, -1.     +0.j,  0.     +0.j],
       [ 0.03355+0.j,  0.91802+0.j, -0.00685+0.j,  0.39505+0.j, -0.     +0.j, -0.     +0.j,  0.     +0.j, -0.     +0.j,  0.     +0.j],
       [ 0.     +0.j,  0.     +0.j, -0.     +0.j, -0.     +0.j, -0.39671+0.j,  0.91794+0.j, -0.     +0.j,  0.     +0.j,  0.     +0.j],
       [-0.     +0.j,  0.     +0.j, -0.     +0.j, -0.     +0.j, -0.91794+0.j, -0.39671+0.j,  0.     +0.j, -0.     +0.j, -0.     +0.j],
       [-0.39277+0.j,  0.01897+0.j,  0.91943+0.j,  0.00

# Optimal reference state in TF-CCC

In [179]:
site = 3
state = 3
g = 0.9

In [180]:
# H, K, V = hamiltonian_general_dense(state, site, g)

# V = V * g

In [181]:
H, K, V = hamiltonian_dense(state, site, g)

V = V * g

In [182]:
K

array([[0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j]])

In [183]:
V

array([[ 0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.675+0.j, -0.225+0.j,  0.   +0.j, -0.225+0.j,  0.675+0.j],
       [ 0.   +0.j,  0.   +0.j,  0.   +0.j, -0.225+0.j,  0.   +0.j,  0.   +0.j,  0.675+0.j,  0.   +0.j,  0.   +0.j],
       [ 0.   +0.j,  0.   +0.j,  0.   +0.j,  0.675+0.j,  0.   +0.j,  0.   +0.j, -0.225+0.j,  0.   +0.j,  0.   +0.j],
       [ 0.   +0.j, -0.225+0.j,  0.675+0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j],
       [ 0.675+0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j],
       [-0.225+0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j],
       [ 0.   +0.j,  0.675+0.j, -0.225+0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j],
       [-0.225+0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j],
       [ 0.675+0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0

In [184]:
state_2 = state**2

In [185]:
U, _ = bz.thermofield_change_of_basis(K)

I = np.eye(state)

K_prim = oe.contract('pq,mw->pmqw', K, I,  optimize='optimal')

K_grouped = K_prim.reshape(state_2, state_2)

V_tensor = V.reshape(state, state, state, state)


V_prim = oe.contract('pqrs,mw,nv->pnqmrvsw', V_tensor, I, I,  optimize='optimal')

V_grouped = V_prim.reshape(state_2**2, state_2**2)

In [186]:
H_TF, _, _ = hamiltonian_dense(state_2, site, 1, K_import=K_grouped, V_import=V_grouped, Import=True)

In [187]:
eig_val_TF, eig_vec_TF = np.linalg.eigh(H_TF)

In [188]:
index_TF = np.argsort(eig_val_TF)
psi_vec = eig_vec_TF[index_TF[0]]

rho_site_0 = density_matrix_1(state_2, site, psi_vec, 0)

In [189]:
eig_val_TF[index_TF]

array([-1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.53639, -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 , -1.3985 ,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.25493,  0.

In [190]:
zero = H_TF[0, :]

In [191]:
np.nonzero(zero)

(array([ 30,  33,  57,  60, 246, 249, 270, 297, 489, 492, 513, 540]),)

In [192]:
ground_NO = eig_vec_TF[:, index_TF[:state ** site]]

In [193]:
(ground_NO.T).real

array([[ 0.     ,  0.     , -0.     , -0.     , -0.     ,  0.     ,  0.     , -0.     ,  0.     ,  0.     ,  0.     , -0.     , -0.     , -0.     , -0.     ,  0.     , -0.     , -0.     , -0.     , -0.     ,  0.     , -0.     , -0.     , -0.     , -0.     ,  0.     ,  0.     ,  0.     , -0.     ,  0.     ,  0.     , -0.     ,  0.     ,  0.     ,  0.     , -0.     , -0.     , -0.     , -0.     , -0.     , -0.     ,  0.     ,  0.     ,  0.     , -0.     ,  0.     , -0.     ,  0.     ,  0.     ,  0.     , -0.     , -0.     , -0.     ,  0.     , -0.     ,  0.     , -0.     ,  0.     ,  0.     , -0.     ,  0.     , -0.     ,  0.     ,  0.     , -0.     , -0.     ,  0.     ,  0.     , -0.     , -0.     , -0.     ,  0.     ,  0.     ,  0.     ,  0.     , -0.     , -0.     ,  0.     ,  0.     ,  0.     , -0.     ,  0.00057, -0.00251, -0.02029, -0.     , -0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.02298,  0.00672, -0.00806,  0.     , -0.     ,  0.     ,  0.     ,  0.     ,  0.     , -0

In [194]:
psi_g = ground_NO @ (ground_NO.conj().T @ zero)

psi_g_norm = psi_g / np.sqrt(psi_g@psi_g)

In [195]:
rho_site_0 = density_matrix_1(state_2, site, psi_g_norm, 0)

In [196]:
ev_d, ec_d = np.linalg.eigh(rho_site_0)

In [197]:
index_D = np.argsort(-ev_d)
vec = ec_d[index_D[0]]

In [198]:
ev_d[index_D]

array([ 0.69628,  0.29252,  0.0112 ,  0.     ,  0.     ,  0.     ,  0.     , -0.     , -0.     ])

In [199]:
ec_d[:, index_D]

array([[ 1.     +0.j, -0.     -0.j,  0.     +0.j, -0.     -0.j, -0.     -0.j,  0.     +0.j,  0.     +0.j,  0.     +0.j,  0.     +0.j],
       [ 0.     +0.j, -0.     +0.j,  0.     +0.j, -0.06343+0.j,  0.     +0.j, -0.     +0.j,  0.     +0.j, -0.     +0.j,  0.99799+0.j],
       [ 0.     +0.j,  0.     +0.j, -0.     +0.j, -0.99799+0.j,  0.     +0.j, -0.     +0.j, -0.     +0.j, -0.     +0.j, -0.06343+0.j],
       [ 0.     +0.j, -0.70711+0.j,  0.70711+0.j, -0.     +0.j, -0.     +0.j, -0.     +0.j, -0.     +0.j,  0.     +0.j, -0.     +0.j],
       [ 0.     +0.j,  0.     +0.j, -0.     +0.j, -0.     +0.j,  0.00001+0.j, -0.     +0.j,  0.     +0.j,  1.     +0.j,  0.     +0.j],
       [ 0.     +0.j, -0.     +0.j,  0.     +0.j, -0.     +0.j,  0.05407+0.j,  0.69493+0.j,  0.71704+0.j, -0.     +0.j,  0.     +0.j],
       [ 0.     +0.j,  0.70711+0.j,  0.70711+0.j, -0.     +0.j,  0.     +0.j, -0.     +0.j, -0.     +0.j, -0.     +0.j, -0.     +0.j],
       [ 0.     +0.j, -0.     +0.j, -0.     +0.j,  0.  

In [200]:
psi_g_norm

array([-0.73785+0.j, -0.     +0.j,  0.     +0.j,  0.     +0.j,  0.     +0.j, -0.     +0.j,  0.     +0.j, -0.     +0.j, -0.     +0.j, -0.     +0.j, -0.     +0.j, -0.     +0.j,  0.     +0.j,  0.     +0.j, -0.     +0.j, -0.     +0.j, -0.     +0.j,  0.     +0.j,  0.     +0.j, -0.     +0.j,  0.     +0.j,  0.     +0.j,  0.     +0.j, -0.     +0.j,  0.     +0.j, -0.     +0.j,  0.     +0.j, -0.     +0.j,  0.     +0.j,  0.     +0.j,  0.22864+0.j, -0.     +0.j,  0.     +0.j, -0.1538 +0.j,  0.     +0.j, -0.     +0.j,  0.     +0.j,  0.     +0.j, -0.     +0.j,  0.     +0.j,  0.     +0.j,  0.     +0.j, -0.     +0.j, -0.     +0.j, -0.     +0.j, -0.     +0.j,  0.     +0.j, -0.     +0.j, -0.     +0.j,  0.     +0.j, -0.     +0.j,  0.     +0.j, -0.     +0.j,  0.     +0.j,  0.     +0.j, -0.     +0.j, -0.     +0.j, -0.1538 +0.j,  0.     +0.j, -0.     +0.j,  0.22864+0.j, -0.     +0.j,  0.     +0.j, -0.     +0.j, -0.     +0.j,  0.     +0.j, -0.     +0.j, -0.     +0.j, -0.     +0.j,  0.     +0.j,  0.     +0.j,

In [201]:
# f_cut = f_norm.reshape(-1, 9)
f_cut = psi_g_norm.reshape(9, 9, 9)
# f_cut = f_cut[:1, :].real

In [202]:
f_cut[:, 0, 0].real

array([-0.73785,  0.     , -0.     , -0.     ,  0.     ,  0.     ,  0.     , -0.     , -0.     ])

In [204]:
mask = (np.all(np.abs(f_cut[:, :3]) > 1e-10, axis=1)) & (np.all(np.abs(f_cut[:, 3:]) < 1e-10, axis=1))

rows = f_cut[mask].T
print(rows)

[]


In [205]:
Q, R = np.linalg.qr(rows[:, :9])

In [1]:
eig_val_i, eig_vec_i = np.linalg.eigh(Q)
eig_val_i

NameError: name 'np' is not defined